In [1]:
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model

# Load the ASL model
loaded_model = load_model('ASL2.h5')

# Define labels dictionary (replace with your actual label mapping)
labels_dict = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'Del',
    5: 'E',
    6: 'F',
    7: 'G',
    8: 'H',
    9: 'I',
    10: 'J',
    11: 'K',
    12: 'L',
    13: 'M',
    14: 'N',
    15: 'Nothing',
    16: 'O',
    17: 'P',
    18: 'Q',
    19: 'R',
    20: 'S',
    21: 'Space',
    22: 'T',
    23: 'U',
    24: 'V',
    25: 'W',
    26: 'X',
    27: 'Y',
    28: 'Z'
}



# Open a connection to the webcam (0 represents the default camera)
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

while True:
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]  # Assuming only one hand is detected

        # Extract coordinates of bounding box around hand
        x_min = int(min(hand_landmarks.landmark[i].x * frame.shape[1] for i in range(21)))
        y_min = int(min(hand_landmarks.landmark[i].y * frame.shape[0] for i in range(21)))
        x_max = int(max(hand_landmarks.landmark[i].x * frame.shape[1] for i in range(21)))
        y_max = int(max(hand_landmarks.landmark[i].y * frame.shape[0] for i in range(21)))

        # Crop the hand region
        hand_roi = frame[y_min:y_max, x_min:x_max]
        hand_roi_resized = cv2.resize(hand_roi, (32, 32))  # Resize to match input size of the model

        # Preprocess the image and make a prediction
        hand_input = np.expand_dims(hand_roi_resized, axis=0) / 255.0  # Normalize to [0, 1]
        prediction = loaded_model.predict(hand_input)

        predicted_character = labels_dict[np.argmax(prediction)]

        cv2.putText(frame, predicted_character, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    cv2.imshow('ASL Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'